# Setup
(No need to read)

In [1]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEBUG_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    # Install another version of node that makes PySvelte work way faster
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    %pip install git+https://github.com/neelnanda-io/PySvelte.git
    # Needed for PySvelte to work, v3 came out and broke things...
    %pip install typeguard==2.13.3
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Colab notebook
  Cloning https://github.com/neelnanda-io/TransformerLens.git to /tmp/pip-req-build-iimyyw6_
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-iimyyw6_
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit 10d2f8a026d73eada861c7d51064f7e24d8f482c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━

In [2]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio

if IN_COLAB or not DEBUG_MODE:
    # Thanks to annoying rendering issues, Plotly graphics will either show up in colab OR Vscode depending on the renderer - this is bad for developing demos! Thus creating a debug mode.
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "png"

In [3]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
from dataclasses import dataclass
import datasets
from IPython.display import HTML

In [4]:
import pysvelte

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

Plotting helper functions:

In [5]:
import plotly.graph_objects as go

update_layout_set = {"xaxis_range", "yaxis_range", "hovermode", "xaxis_title", "yaxis_title", "colorbar", "colorscale", "coloraxis", "title_x", "bargap", "bargroupgap", "xaxis_tickformat", "yaxis_tickformat", "title_y", "legend_title_text", "xaxis_showgrid", "xaxis_gridwidth", "xaxis_gridcolor", "yaxis_showgrid", "yaxis_gridwidth"}
def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    if isinstance(tensor, list):
        tensor = torch.stack(tensor)
    kwargs_post = {k: v for k, v in kwargs.items() if k in update_layout_set}
    kwargs_pre = {k: v for k, v in kwargs.items() if k not in update_layout_set}
    if "facet_labels" in kwargs_pre:
        facet_labels = kwargs_pre.pop("facet_labels")
    else:
        facet_labels = None
    if "color_continuous_scale" not in kwargs_pre:
        kwargs_pre["color_continuous_scale"] = "RdBu"
    fig = px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0,labels={"x":xaxis, "y":yaxis}, **kwargs_pre).update_layout(**kwargs_post)
    if facet_labels:
        for i, label in enumerate(facet_labels):
            fig.layout.annotations[i]['text'] = label

    fig.show(renderer)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(y=utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

def lines(lines_list, x=None, mode='lines', labels=None, xaxis='', yaxis='', title = '', log_y=False, hover=None, **kwargs):
    # Helper function to plot multiple lines
    if type(lines_list)==torch.Tensor:
        lines_list = [lines_list[i] for i in range(lines_list.shape[0])]
    if x is None:
        x=np.arange(len(lines_list[0]))
    fig = go.Figure(layout={'title':title})
    fig.update_xaxes(title=xaxis)
    fig.update_yaxes(title=yaxis)
    for c, line in enumerate(lines_list):
        if type(line)==torch.Tensor:
            line = utils.to_numpy(line)
        if labels is not None:
            label = labels[c]
        else:
            label = c
        fig.add_trace(go.Scatter(x=x, y=line, mode=mode, name=label, hovertext=hover, **kwargs))
    if log_y:
        fig.update_layout(yaxis_type="log")
    fig.show()

def bar(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.bar(
        y=utils.to_numpy(tensor),
        labels={"x": xaxis, "y": yaxis},
        template="simple_white",
        **kwargs).show(renderer)

In [6]:
import transformer_lens.patching as patching
from transformer_lens import evals
import math

In [7]:
def visualize_attn_patterns(heads, local_tokens, local_cache, title: str = ""):
    labels = []
    patterns = []
    batch_index = 0

    for head in heads:
        if isinstance(head, tuple):
            layer, head_index = head
        else:
            layer, head_index = head // model.cfg.n_heads, head % model.cfg.n_heads
        patterns.append(local_cache["pattern", layer][batch_index, head_index])
        labels.append(f"L{layer}H{head_index}")
    patterns = torch.stack(patterns, dim=-1)
    attn_viz = pysvelte.AttentionMulti(tokens=model.to_str_tokens(local_tokens[batch_index]), attention=patterns, head_labels=labels)
    display(HTML(f"<h3>{title}</h3>"))
    attn_viz.show()

# Load Model

In [8]:
torch.set_grad_enabled(True)

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [10]:
p = 113
@dataclass
class Config:
    d_model: int = 128
    d_mlp: int = 512
    d_vocab: int = p
    d_vocab_out: int = p
    device: int = device
    seed: int = 0

cfg = Config()
print(cfg)

Config(d_model=128, d_mlp=512, d_vocab=113, d_vocab_out=113, device='cuda', seed=0)


In [11]:
class OneLayerMLP(HookedRootModule):
    def __init__(self, cfg):
        super().__init__()
        torch.manual_seed(cfg.seed)

        self.W_E = nn.Parameter(torch.randn(cfg.d_vocab, cfg.d_model) / np.sqrt(cfg.d_model))
        self.W_in = nn.Parameter(torch.randn(cfg.d_model, cfg.d_mlp) / np.sqrt(cfg.d_model))
        self.W_U = nn.Parameter(torch.randn(cfg.d_mlp, cfg.d_vocab_out) / np.sqrt(cfg.d_mlp))

        self.hook_embed = HookPoint()
        self.hook_pre = HookPoint()
        self.hook_post = HookPoint()

        super().setup()

    def forward(self, data):
        a = data[:, 0]
        b = data[:, 1]
        embed = self.hook_embed(self.W_E[a] + self.W_E[b])
        pre = self.hook_pre(embed @ self.W_in)
        post = self.hook_post(F.relu(pre))
        logits = post @ self.W_U
        return logits

model = OneLayerMLP(cfg).to(cfg.device)
print(model)

OneLayerMLP(
  (hook_embed): HookPoint()
  (hook_pre): HookPoint()
  (hook_post): HookPoint()
)


# Training

## Task dataset

In [12]:
a_vec = einops.repeat(torch.arange(p), "i -> (i j)", j=p)
print(a_vec.shape)
print(a_vec[:5])

b_vec = einops.repeat(torch.arange(p), "i -> (j i)", j=p)
print(b_vec.shape)
print(b_vec[:5])

dataset = torch.stack([a_vec, b_vec], dim=-1).to(device)
print(dataset.shape)
print(dataset)

torch.Size([12769])
tensor([0, 0, 0, 0, 0])
torch.Size([12769])
tensor([0, 1, 2, 3, 4])
torch.Size([12769, 2])
tensor([[  0,   0],
        [  0,   1],
        [  0,   2],
        ...,
        [112, 110],
        [112, 111],
        [112, 112]], device='cuda:0')


In [13]:
labels = (dataset[:, 0] + dataset[:, 1]) % p
print(labels.shape)
print(labels[:5])

torch.Size([12769])
tensor([0, 1, 2, 3, 4], device='cuda:0')


In [14]:
DATA_SEED = 0
torch.manual_seed(DATA_SEED)

train_frac = 0.3
cutoff = int(train_frac * dataset.shape[0])

indices = torch.randperm(dataset.shape[0])

train_indices = indices[:cutoff]
train_data = dataset[train_indices]
train_labels = labels[train_indices]
print(train_data.shape)
print(train_data[:5])
print(train_labels.shape)
print(train_labels[:5])

test_indices = indices[cutoff:]
test_data = dataset[test_indices]
test_labels = labels[test_indices]
print(test_data.shape)
print(test_data[:5])
print(test_labels.shape)
print(test_labels[:5])

torch.Size([3830, 2])
tensor([[ 37,   1],
        [ 23,   9],
        [  2,  43],
        [  7,  34],
        [ 42, 101]], device='cuda:0')
torch.Size([3830])
tensor([38, 32, 45, 41, 30], device='cuda:0')
torch.Size([8939, 2])
tensor([[ 55,  55],
        [ 69, 109],
        [ 13,  17],
        [103,  54],
        [ 78, 112]], device='cuda:0')
torch.Size([8939])
tensor([110,  65,  30,  44,  77], device='cuda:0')


## Loss fn

In [15]:
def loss_fn(logits, labels):
    if logits.ndim == 3:
        logits = logits[:, -1, :]
    logits = logits.to(torch.float64)
    log_probs = logits.log_softmax(dim=-1)
    correct_log_probs = log_probs.gather(dim=-1, index=labels[:, None])
    return -correct_log_probs.mean()

with torch.no_grad():
    train_logits = model(train_data)
    train_loss = loss_fn(train_logits, train_labels)
    print(train_loss)

    test_logits = model(test_data)
    test_loss = loss_fn(test_logits, test_labels)
    print(test_loss)

tensor(4.7313, device='cuda:0', dtype=torch.float64)
tensor(4.7308, device='cuda:0', dtype=torch.float64)


In [16]:
print("uniform loss:", np.log(cfg.d_vocab_out))

uniform loss: 4.727387818712341


## Setup optimizer

In [17]:
lr = 1e-3
wd = 1.0
betas = (0.9, 0.95)

optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd, betas=betas)

## Training Loop

In [18]:
num_epochs = 50000

train_losses = []
test_losses = []
for epoch in tqdm.tqdm(range(num_epochs)):
    train_logits = model(train_data)
    train_loss = loss_fn(train_logits, train_labels)
    train_loss.backward()
    train_losses.append(train_loss.item())

    optimizer.step()
    optimizer.zero_grad()

    with torch.inference_mode():
        test_logits = model(test_data)
        test_loss = loss_fn(test_logits, test_labels)
        test_losses.append(test_loss.item())

    if epoch % 100 == 0:
        print(f"Epoch: {epoch}, train_loss: {train_loss.item()}, test_loss: {test_loss.item()}")

  0%|          | 0/50000 [00:00<?, ?it/s]

Epoch: 0, train_loss: 4.731251124626039, test_loss: 4.73271321178515
Epoch: 100, train_loss: 3.2684520377331023, test_loss: 5.77628146499774
Epoch: 200, train_loss: 1.0439769383865525, test_loss: 7.1238887720093675
Epoch: 300, train_loss: 0.06124430475210127, test_loss: 8.803239889885937
Epoch: 400, train_loss: 0.005048883034357713, test_loss: 9.714085323880251
Epoch: 500, train_loss: 0.0004721027763956813, test_loss: 10.249435731001139
Epoch: 600, train_loss: 4.8561741648956785e-05, test_loss: 10.503716844033292
Epoch: 700, train_loss: 7.477306768130827e-06, test_loss: 10.384872494585064
Epoch: 800, train_loss: 3.0055418444505576e-06, test_loss: 9.754051035573022
Epoch: 900, train_loss: 2.5005550685612088e-06, test_loss: 8.85793307323366
Epoch: 1000, train_loss: 2.422806893620187e-06, test_loss: 7.963611699271327
Epoch: 1100, train_loss: 2.3765889178209445e-06, test_loss: 7.113391884725417
Epoch: 1200, train_loss: 2.329921933186787e-06, test_loss: 6.3042901707819565
Epoch: 1300, train

In [19]:
lines(
    [train_losses, test_losses],
    title="Loss curves",
    xaxis="Epoch", yaxis="Loss",
    labels=["Train", "Test"]
)

## Sanity check

In [20]:
with torch.inference_mode():
    test_logits = model(test_data)
    preds = test_logits.argmax(dim=-1)
    test_acc = (preds == test_labels).float().mean()
    print("Test accuracy:", test_acc.item())

Test accuracy: 1.0


# Save model

In [21]:
%mkdir ../models

In [22]:
filename = "../models/modular_addition_2l_mlp_model.pt"
torch.save(model.state_dict(), filename)

In [23]:
# sanity check we can load model
loaded_model = OneLayerMLP(cfg).to(cfg.device)
loaded_model.load_state_dict(torch.load(filename), strict=True)

<All keys matched successfully>